## ライブラリ

In [7]:
# 数値計算に使うライブラリインポート
import numpy as np
import pandas as pd

# DataFrameの全角文字の出力をきれいにする
pd.set_option("display.unicode.east_asian_width", True)

## 利得行列

In [8]:
# パラメータ
fixed_cost = 100
run_cost = 600
sale_price = 0.2
machine_ability = 5000
demand_boom = 10000
demand_slump = 5000

In [9]:
# 出荷される製品の個数
num_product_df = pd.DataFrame({
    "0台": [0, 0], 
    "1台": [min([machine_ability, demand_boom]), min([machine_ability, demand_slump])], 
    "2台": [min([machine_ability * 2, demand_boom]), min([machine_ability * 2, demand_slump])]
})
num_product_df.index = ["好況", "不況"]
num_product_df

,0台,1台,2台
好況,0,5000,10000
不況,0,5000,5000


In [10]:
# 売上行列
sales_df = num_product_df * sale_price
sales_df

,0台,1台,2台
好況,0.0,1000.0,2000.0
不況,0.0,1000.0,1000.0


In [11]:
# 製造コスト
run_cost_df = pd.DataFrame({
    "0台": np.repeat(fixed_cost, 2), 
    "1台": np.repeat(fixed_cost + run_cost, 2), 
    "2台": np.repeat(fixed_cost + 2 * run_cost, 2)
})
run_cost_df.index = ["好況", "不況"]
run_cost_df

,0台,1台,2台
好況,100,700,1300
不況,100,700,1300


In [12]:
# 利得行列
payoff_df = sales_df - run_cost_df
payoff_df

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0


In [13]:
# パラメータを引数として利得行列を返す関数を作成
def calc_payoff_table(fixed_cost, run_cost, sale_price, machine_ability, demand_boom, demand_slump):

  # 出荷される製品の個数
  num_product_df = pd.DataFrame({
    "0台": [0, 0], 
    "1台": [min([machine_ability, demand_boom]), min([machine_ability, demand_slump])], 
    "2台": [min([machine_ability * 2, demand_boom]), min([machine_ability * 2, demand_slump])]
  })

  # 売上行列
  sales_df = num_product_df * sale_price

  # 製造コスト
  run_cost_df = pd.DataFrame({
      "0台": np.repeat(fixed_cost, 2), 
      "1台": np.repeat(fixed_cost + run_cost, 2), 
      "2台": np.repeat(fixed_cost + 2 * run_cost, 2)
  })

  # 利得行列
  payoff_df = sales_df - run_cost_df
  payoff_df.index = ["好況", "不況"]

  # 利得行列を出力
  return payoff_df

In [14]:
# 上記の関数を用いて利得行列を作成
payoff = calc_payoff_table(fixed_cost = 100, run_cost = 600, sale_price = 0.2, machine_ability = 5000, demand_boom = 10000, demand_slump = 5000)
payoff

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0


## マキシマックス基準

In [15]:
# 選択肢ごとの最大利得
payoff.max(axis = 0)

0台   -100.0
1台    300.0
2台    700.0
dtype: float64

In [16]:
# 最大利得を最大にするのがマキシマックス基準
payoff.max(axis = 0).max(axis = 0)

700.0

In [17]:
# 最大利得を得る選択肢（インデックス）を取得
# ただし，idxmax()は最大値が複数ある場合，最初のインデックスのみ返す
payoff.max().idxmax()

'2台'

In [18]:
# 最大値と等しい利得があるかどうかを確認（今回は，最大値を取る選択肢が複数ある場合は列挙する形にする）
# payoff.max() = [-100, 300, 700]の配列にpayoff.max().max() = 700があるかどうかを確認
payoff.max() == payoff.max().max()

0台    False
1台    False
2台     True
dtype: bool

In [19]:
# 最大値と等しい利得を持つ要素を取得
# 上記でTrueの要素だけ抽出
payoff.max()[payoff.max() == payoff.max().max()]

2台    700.0
dtype: float64

In [20]:
# 欲しいのは選択肢（＝インデックス）のみ
list(payoff.max()[payoff.max() == payoff.max().max()].index)

['2台']

In [21]:
# 最大値をとるインデックスをリストで取得する関数を作成（最大値を取る選択肢が複数ある場合は列挙する形にする）
# 上記のpayoff.max()をseriesで置き換え
def argmax_list(series):
  return list(series[series == series.max()].index)

In [22]:
# 最小値をとるインデックスをリストで取得する関数を作成（最小値を取る選択肢が複数ある場合は列挙する形にする）
def argmin_list(series):
  return list(series[series == series.min()].index)

In [23]:
# マキシマックス基準による意思決定
print("Maximax:", argmax_list(payoff.max()))

Maximax: ['2台']


## マキシミン基準

In [24]:
# 選択肢ごとの最小利得
payoff.min(axis = 0)

0台   -100.0
1台    300.0
2台   -300.0
dtype: float64

In [25]:
# 最小利得を最大にするのがマキシミン基準
print("Maximin:", argmax_list(payoff.min()))

Maximin: ['1台']


## ハーヴィッツの基準

In [26]:
# 楽観係数
alpha = 0.6

# 選択肢ごとに最大利得と最小利得を楽観係数で重み付けした合計を計算
hurwicz = payoff.max() * alpha + payoff.min() * (1 - alpha)
hurwicz

0台   -100.0
1台    300.0
2台    300.0
dtype: float64

In [27]:
# 上記を最大にするのがハーヴィッツの基準
# 1台と2台がindifferent
argmax_list(hurwicz)

['1台', '2台']

In [28]:
# 関数にまとめる
def hurwicz(payoff_table, alpha):
  hurwicz = payoff_table.max() * alpha + payoff_table.min() * (1 - alpha)
  return argmax_list(hurwicz)

In [29]:
# ハーヴィッツの基準による意思決定
print("Hurwicz(alpha = 0.6):", hurwicz(payoff, 0.6))

Hurwicz(alpha = 0.6): ['1台', '2台']


In [30]:
# alphaの比較静学：より楽観的に
print("Hurwicz(alpha = 0.7):", hurwicz(payoff, 0.7))

Hurwicz(alpha = 0.7): ['2台']


In [31]:
# alphaの比較静学：より悲観的に
print("Hurwicz(alpha = 0.5):", hurwicz(payoff, 0.5))

Hurwicz(alpha = 0.5): ['1台']


## ミニマックスリグレット基準

In [32]:
# 自然の状態ごとに見た最大利得
# pd.concat()：DataFrame, Seriesを連結する
best_df = pd.concat([payoff.max(axis = 1)] * payoff.shape[1], axis = 1)
best_df.columns = payoff.columns
best_df

,0台,1台,2台
好況,700.0,700.0,700.0
不況,300.0,300.0,300.0


In [33]:
# リグレット行列
regret_df = best_df - payoff
regret_df

,0台,1台,2台
好況,800.0,400.0,0.0
不況,400.0,0.0,600.0


In [34]:
# 選択肢ごとのリグレットの最大値を取得
regret_df.max(axis = 0)

0台    800.0
1台    400.0
2台    600.0
dtype: float64

In [35]:
# 上記を最小にするのがミニマックスリグレット基準
argmin_list(regret_df.max())

['1台']

In [36]:
# 関数にまとめる
def minimax_regret(payoff_table):
  
  # 自然の状態ごとの最大利得
  best_df = pd.concat([payoff_table.max(axis = 1)] * payoff_table.shape[1], axis = 1)
  best_df.columns = payoff_table.columns

  # リグレット行列
  regret_df = best_df - payoff_table

  # 出力
  return argmin_list(regret_df.max())

In [37]:
# ミニマックスリグレット基準による意思決定
print("Minimax regret:", minimax_regret(payoff))

Minimax regret: ['1台']


## ラプラスの基準

In [38]:
# 選択肢ごとの平均利得
payoff.mean(axis = 0)

0台   -100.0
1台    300.0
2台    200.0
dtype: float64

In [39]:
# 上記を最大にするのがラプラスの基準
print("Laplace:", argmax_list(payoff.mean()))

Laplace: ['1台']


## 感度分析：ミニマックスリグレット基準を例に

In [40]:
# 機械1台の稼働コストを25万円増やしたときの利得行列
payoff_2 = calc_payoff_table(fixed_cost = 100, run_cost = 625, sale_price = 0.2, machine_ability = 5000, demand_boom = 10000, demand_slump = 5000)
payoff_2

,0台,1台,2台
好況,-100.0,275.0,650.0
不況,-100.0,275.0,-350.0


In [41]:
# ミニマックスリグレット基準による意思決定
print("Minimax regret:", minimax_regret(payoff_2))

Minimax regret: ['1台']


In [42]:
# 機械1台の稼働コストを25万円減らしたときの利得行列
payoff_3 = calc_payoff_table(fixed_cost = 100, run_cost = 575, sale_price = 0.2, machine_ability = 5000, demand_boom = 10000, demand_slump = 5000)
payoff_3

,0台,1台,2台
好況,-100.0,325.0,750.0
不況,-100.0,325.0,-250.0


In [43]:
# ミニマックスリグレット基準による意思決定
print("Minimax regret:", minimax_regret(payoff_3))

Minimax regret: ['1台']


In [44]:
# 機械1台の稼働コストを100万円減らしたときの利得行列
payoff_4 = calc_payoff_table(fixed_cost = 100, run_cost = 500, sale_price = 0.2, machine_ability = 5000, demand_boom = 10000, demand_slump = 5000)
payoff_4

,0台,1台,2台
好況,-100.0,400.0,900.0
不況,-100.0,400.0,-100.0


In [45]:
# ミニマックスリグレット基準による意思決定
print("Minimax regret:", minimax_regret(payoff_4))

Minimax regret: ['1台', '2台']
